In [50]:
#### SET UP ####
# Title: Classroom formulas V4
# Auteur: Astrid Lensink
# Last modified on: 31-10-2024
# Python-version

In [51]:
#### Import libraries ####
import pandas as pd
import numpy as np
from openpyxl import load_workbook  
from IPython.display import display, HTML

In [52]:
#### Load data ####
data_path = "C:/Users/AstridLensink/SEO/001025 Reward Value Experimenten - Documenten/2 Classroom experiment/Analyse/Classroom_V4_input.xlsx"
data = pd.read_excel(data_path)

# Display first rows
data.head()

,t,L_1,L_2,L_3,K_1,K_2,E_2,RD_L,RD_K,RD_E,...,points_profit,points_RD,points_ESG,input_wage,input_labour,input_delta_K,input_E,input_delta_RD_L,input_delta_RD_K,input_delta_RD_E
0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4,50,10,10,20,20,20
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4,60,20,10,20,20,20
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4,60,30,15,20,20,20
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4,70,30,30,40,40,40


In [53]:
##### Constants #####

# Budget
budget = 1000

# Tax rate and tax exemption
tax_rate = 0.5
free_budget = 1000

# Investment of profit to budget in t+1
percent_investment = 0.2

# Minimum wage
W_min = 4

# Alphas
wage_alpha = 40
capital_alpha = 1
energy_alpha = 2
output_alpha = 1

# Betas
wage_beta = 2 / 3
labour_beta = 1 / 3
capital_beta = 1 / 3
energy_beta = 1 / 3

# Prices
P_K = 15
P_E = 10
P_O = 80
P_RD_L = 5
P_RD_K = 5
P_RD_E = 5

# Depreciation
depreciation_K = 0.1
depreciation_RD_L = 0.2
depreciation_RD_K = 0.2
depreciation_RD_E = 0.2

# Points
## General points
point_profit = 0.05
point_RD = 1.5
point_social = 2
point_eco = -2

## Treatment points
point_profit_treat = 0.15
point_RD_treat = 1.5
point_social_treat = 4
point_eco_treat = -3

In [54]:
#### Functions ####

# Determine maximum of labour possible
def fun_labour_max(input_wage):
    L_1 = max(wage_alpha * (input_wage ** wage_beta) - wage_alpha, 0)
    return L_1

# Set amount labour hired
def fun_labour_chosen(input_labour, L_1):
    return min(input_labour, L_1)

# Determine the capital stock
def fun_capital(input_delta_K, K):
    return (1 - depreciation_K) * K + input_delta_K

# Determine R&D stock
def fun_RD(RD, delta_RD, depreciation):
    return (1 - depreciation) * RD + delta_RD / 100

# Update capital, energy and labour productivity by RD stock
def fun_update_RD(input_val, RD):
    return input_val * np.sqrt(RD)

# Calculate social externalities 
def fun_social(input_wage, W_min, L_2):
    return (input_wage - W_min) * L_2

# Calculate environmental externalities
def fun_eco(capital_alpha, K_1, energy_alpha, input_E):
    return capital_alpha * K_1 + energy_alpha * input_E


In [55]:
#### Processing the data ####
data.loc[1, 'budget'] = budget

for t in range(1, len(data)):
    # Labour
    data.loc[t, 'L_1'] = fun_labour_max(data.loc[t, 'input_wage'])
    data.loc[t, 'L_2'] = fun_labour_chosen(data.loc[t, 'input_labour'], data.loc[t, 'L_1'])
    data.loc[t, 'C_L'] = data.loc[t, 'L_2'] * data.loc[t, 'input_wage']

    # Capital
    data.loc[t, 'K_1'] = fun_capital(data.loc[t, 'input_delta_K'], data.loc[t - 1, 'K_1'])
    data.loc[t, 'C_K'] = data.loc[t, 'input_delta_K'] * P_K

    # Energy
    data.loc[t, 'C_E'] = data.loc[t, 'input_E'] * P_E

    # R&D
    data.loc[t, 'RD_L'] = fun_RD(data.loc[t - 1, 'RD_L'], data.loc[t, 'input_delta_RD_L'], depreciation_RD_L)
    data.loc[t, 'RD_K'] = fun_RD(data.loc[t - 1, 'RD_K'], data.loc[t, 'input_delta_RD_K'], depreciation_RD_K)
    data.loc[t, 'RD_E'] = fun_RD(data.loc[t - 1, 'RD_E'], data.loc[t, 'input_delta_RD_E'], depreciation_RD_E)
    data.loc[t, 'C_RD_L'] = P_RD_L * data.loc[t, 'input_delta_RD_L']
    data.loc[t, 'C_RD_K'] = P_RD_K * data.loc[t, 'input_delta_RD_K']
    data.loc[t, 'C_RD_E'] = P_RD_E * data.loc[t, 'input_delta_RD_E']

    # Output R&D
    data.loc[t, 'L_3'] = fun_update_RD(data.loc[t, 'L_2'], data.loc[t, 'RD_L'])
    data.loc[t, 'K_2'] = fun_update_RD(data.loc[t, 'K_1'], data.loc[t, 'RD_K'])
    data.loc[t, 'E_2'] = fun_update_RD(data.loc[t, 'input_E'], data.loc[t, 'RD_E'])

    # Output
    data.loc[t, 'O'] = output_alpha * (data.loc[t, 'L_3'] ** labour_beta) * (data.loc[t, 'K_2'] ** capital_beta) * (data.loc[t, 'E_2'] ** energy_beta)
    data.loc[t, 'R'] = data.loc[t, 'O'] * P_O
    data.loc[t, 'C'] = data.loc[t, 'C_L'] + data.loc[t, 'C_K'] + data.loc[t, 'C_E'] + data.loc[t, 'C_RD_L'] + data.loc[t, 'C_RD_K'] + data.loc[t, 'C_RD_E']
    
    # Profit calculation
    data.loc[t, 'P'] = (1 - tax_rate) * (data.loc[t, 'R'] - data.loc[t, 'C'] - free_budget) + free_budget

    # Externalities
    data.loc[t, 'S'] = fun_social(data.loc[t, 'input_wage'], W_min, data.loc[t, 'L_2'])
    data.loc[t, 'eco'] = fun_eco(capital_alpha, data.loc[t, 'K_1'], energy_alpha, data.loc[t, 'input_E'])

    # Budget for next period
    if data.loc[t, 'budget'] < data.loc[t, 'C']:
        print("This is not possible")
    if t < 8:
        data.loc[t + 1, 'budget'] = max(data.loc[t, 'budget'] + percent_investment * data.loc[t, 'P'] + (data.loc[t, 'budget'] - data.loc[t, 'C']), budget)

    # Points calculation
    data.loc[t, 'points_base'] = point_profit * data.loc[t, 'P'] + point_RD * (data.loc[t, 'input_delta_RD_E'] + data.loc[t, 'input_delta_RD_K'] + data.loc[t, 'input_delta_RD_E']) + point_social * data.loc[t, 'S'] + point_eco * data.loc[t, 'eco']
    data.loc[t, 'points_profit'] = data.loc[t, 'points_base'] + point_profit_treat * data.loc[t, 'P']
    data.loc[t, 'points_RD'] = data.loc[t, 'points_base'] + point_RD_treat * (data.loc[t, 'input_delta_RD_L'] + data.loc[t, 'input_delta_RD_K'] + data.loc[t, 'input_delta_RD_E'])
    data.loc[t, 'points_ESG'] = data.loc[t, 'points_base'] + point_social_treat * data.loc[t, 'S'] + point_eco_treat * data.loc[t, 'eco']

# Display first rows
data.head()


,t,L_1,L_2,L_3,K_1,K_2,E_2,RD_L,RD_K,RD_E,...,points_profit,points_RD,points_ESG,input_wage,input_labour,input_delta_K,input_E,input_delta_RD_L,input_delta_RD_K,input_delta_RD_E
0,0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
1,1,60.793684,50.000000,50.000000,10.00,10.000000,10.000000,1.0,1.0,1.0,...,191.798076,160.449519,-19.550481,4,50,10,10,20,20,20
2,2,60.793684,60.000000,60.000000,29.00,29.000000,10.000000,1.0,1.0,1.0,...,205.303390,135.325847,-101.674153,4,60,20,10,20,20,20
3,3,60.793684,60.000000,60.000000,56.10,56.100000,15.000000,1.0,1.0,1.0,...,199.482152,78.220538,-270.079462,4,60,30,15,20,20,20
4,4,60.793684,60.793684,66.596144,80.49,88.172377,32.863353,1.2,1.2,1.2,...,301.999615,179.764904,-421.705096,4,70,30,30,40,40,40


In [56]:
#### Display all rows in the DataFrame ####
display(HTML(data.to_html()))

,t,L_1,L_2,L_3,K_1,K_2,E_2,RD_L,RD_K,RD_E,C_L,C_K,C_E,C_RD_L,C_RD_K,C_RD_E,O,R,C,P,S,eco,budget,points_base,points_profit,points_RD,points_ESG,input_wage,input_labour,input_delta_K,input_E,input_delta_RD_L,input_delta_RD_K,input_delta_RD_E
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,1.00000,1.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
1,1,60.793684,50.000000,50.000000,10.000000,10.000000,10.000000,1.00000,1.00000,1.00000,200.000000,150.0,100.0,100.0,100.0,100.0,17.099759,1367.980757,750.000000,808.990379,0.0,30.000000,1000.000000,70.449519,191.798076,160.449519,-19.550481,4,50,10,10,20,20,20
2,2,60.793684,60.000000,60.000000,29.000000,29.000000,10.000000,1.00000,1.00000,1.00000,240.000000,300.0,100.0,100.0,100.0,100.0,25.912924,2073.033895,940.000000,1066.516948,0.0,49.000000,1411.798076,45.325847,205.303390,135.325847,-101.674153,4,60,20,10,20,20,20
3,3,60.793684,60.000000,60.000000,56.100000,56.100000,15.000000,1.00000,1.00000,1.00000,240.000000,450.0,150.0,100.0,100.0,100.0,36.960269,2956.821521,1140.000000,1408.410761,0.0,86.100000,2096.899541,-11.779462,199.482152,78.220538,-270.079462,4,60,30,15,20,20,20
4,4,60.793684,60.793684,66.596144,80.490000,88.172377,32.863353,1.20000,1.20000,1.20000,243.174736,450.0,300.0,200.0,200.0,200.0,57.787136,4622.970881,1593.174736,2014.898073,0.0,140.490000,3335.481234,-0.235096,301.999615,179.764904,-421.705096,4,70,30,30,40,40,40
5,5,76.960710,70.000000,81.633327,82.441000,96.141901,5.830952,1.36000,1.36000,1.36000,350.000000,150.0,50.0,200.0,200.0,200.0,35.768980,2861.518379,1150.000000,1355.759190,70.0,92.441000,5480.767347,202.905959,406.269838,382.905959,205.582959,5,70,10,5,40,40,40
6,6,92.077090,90.000000,116.930749,79.196900,102.895031,12.992305,1.68800,1.68800,1.68800,540.000000,75.0,100.0,300.0,300.0,300.0,53.868695,4309.495571,1615.000000,1847.247786,180.0,99.196900,10082.686531,523.968589,801.055757,793.968589,946.377889,6,90,5,10,60,60,60
7,7,106.372228,100.000000,139.656722,111.277210,155.406104,27.931344,1.95040,1.95040,1.95040,700.000000,600.0,200.0,300.0,300.0,300.0,84.633166,6770.653248,2400.000000,2685.326624,300.0,151.277210,18919.822620,701.711911,1104.510905,971.711911,1447.880281,7,100,40,20,60,60,60
8,8,120.000000,120.000000,176.376325,130.149489,191.294071,14.698027,2.16032,2.16032,2.16032,960.000000,450.0,100.0,300.0,300.0,300.0,79.152921,6332.233698,2410.000000,2461.116849,480.0,150.149489,35976.710564,1052.756864,1421.924392,1322.756864,2522.308397,8,120,30,10,60,60,60


In [57]:
#### Result ####
# Summing points over all periods
uitslag = data[['points_base', 'points_profit', 'points_RD', 'points_ESG']].sum().reset_index()
uitslag.columns = ['treatment', 'points']

# Sorting and printing
uitslag = uitslag.sort_values(by='points', ascending=False)
print(uitslag)

       treatment       points
1  points_profit  4632.344124
3     points_ESG  4309.140335
2      points_RD  4025.104132
0    points_base  2585.104132


In [58]:
#### Show share of points in total ####
points_df = pd.DataFrame({
    'Profit Points': [point_profit * data['P'].sum()],
    'R&D Points': [point_RD * (data['input_delta_RD_E'].sum() + data['input_delta_RD_K'].sum() + data['input_delta_RD_E'].sum())],
    'ESG Points': [point_social * data['S'].sum() + point_eco * data['eco'].sum()]
})


# Calculate total points
total_points = points_df.sum(axis=1).iloc[0]
print(total_points)

# Add row to show the percentage
percentage_row = (points_df / total_points) * 100
percentage_row.index = ['Percentage (%)']

# Append the percentage row 
points_df = pd.concat([points_df, percentage_row], axis=0)

# Display the points share in base points
print(points_df)


2585.1041323835298
                Profit Points   R&D Points  ESG Points
0                  682.413330  1440.000000  462.690802
Percentage (%)      26.397905    55.703752   17.898343
